In [2]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

In [8]:
query_simtest = """
SELECT
  trim('"' FROM block::text )::int as block,
  trim('"' FROM min::text )::int as min,
  s.holding,
  s.acks,
  s.msgqueue,
  s.inmsgqueue,
  s.apiqueue,
  s.ackqueue,
  s.timermsg,
  ts::text::time,
  s.run
FROM (SELECT
  --mt     | "LIST_SIZES Holding: %v, Acks: %v, MsgQueue: %v, InMsgQueue: %v, APIQueue: %v, AckQueue: %v, TimerMsg: %v "
  run,
  e->'log'->'height' as block,
  e->'log'->'min' as min,
  e->'log'->'event'->0 as holding,
  e->'log'->'event'->1 as acks,
  e->'log'->'event'->2 as msgqueue,
  e->'log'->'event'->3 as inmsgqueue,
  e->'log'->'event'->4 as apiqueue,
  e->'log'->'event'->5 as ackqueue,
  e->'log'->'event'->6 as timermsg,
  e->'log'->'event'->7 as fmt,
  e->'ts' as ts
  --e as l
FROM
  logs
) as s

WHERE
  s.fmt::varchar like '%LIST_SIZES%'
"""

query_order = """
ORDER BY ts;
"""


def query_run(i):
    return query_simtest + ' AND run = %s ' % i + query_order

connection = pg.connect("host='localdb' dbname=load user=load password='load'")


In [11]:
d1 = pd.read_sql_query(query_run(1),con=connection)
d2 = pd.read_sql_query(query_run(2),con=connection)
d3 = pd.read_sql_query(query_run(3),con=connection)
d4 = pd.read_sql_query(query_run(4),con=connection)
d5 = pd.read_sql_query(query_run(5),con=connection)
d6 = pd.read_sql_query(query_run(6),con=connection)
d7 = pd.read_sql_query(query_run(7),con=connection)
d8 = pd.read_sql_query(query_run(8),con=connection)
d9 = pd.read_sql_query(query_run(9),con=connection)
d10 = pd.read_sql_query(query_run(10),con=connection)
d11 = pd.read_sql_query(query_run(11),con=connection)
d12 = pd.read_sql_query(query_run(12),con=connection)


In [13]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd
from datetime import datetime


data = [
    go.Scatter(x=d1['ts'], y=d1['holding'], name="LF_d0"),
    go.Scatter(x=d2['ts'], y=d2['holding'], name="LF_d100"),
    go.Scatter(x=d3['ts'], y=d3['holding'], name="LLF_d0"),
    go.Scatter(x=d4['ts'], y=d4['holding'], name="LLF_d100"),
    go.Scatter(x=d5['ts'], y=d5['holding'], name="LLLF_d0"),
    go.Scatter(x=d6['ts'], y=d6['holding'], name="LLLF_d100"),
    
    go.Scatter(x=d7['ts'], y=d7['holding'], name="4LF_d0"),
    go.Scatter(x=d8['ts'], y=d8['holding'], name="4LF_d100"),
    go.Scatter(x=d9['ts'], y=d9['holding'], name="5LF_d100"),
    go.Scatter(x=d10['ts'], y=d10['holding'], name="6LF_d100"),
    go.Scatter(x=d11['ts'], y=d11['holding'], name="7LF_d0"),
    go.Scatter(x=d12['ts'], y=d12['holding'], name="8LF_d100"),

]

py.iplot(data, filename = 'v6.0.2_rc holding queue size by set')